In [ ]:
import pandas as pd
import mytools,os
from geographiclib.geodesic import Geodesic
from shapely.geometry import Polygon,Point,LineString,MultiPolygon,MultiLineString 
import geopandas as gpd
from shapely.ops import unary_union

# 第五版本覆盖空洞
### 使用天线方向模型进行修正
使用农村1500距离和1800距离为半径
### 使用每十度一个扇形修正
使用270-90度的方向图层为有效数据

In [ ]:
#导入工参数据
gongcan = pd.read_csv(open('G:/1-规划/评估/覆盖空洞/20200712第四版本/1-基础数据/ltejizhan20200616&微站_合并.csv',encoding='gbk'),usecols=[
    '小区CGI(*)','所属地区','小区中文名','RRU经度', 'RRU纬度','天线挂高','工作频段', '方位角','归属区县','厂家名称' ])
#设置变量通用名
print('设置变量通用名')
id = '小区CGI(*)'
lon = 'RRU经度'
lat = 'RRU纬度'
zhishi = '工作频段'
fw = '方位角'

gongcan = gongcan.loc[(gongcan['RRU经度']>108)&
            (gongcan['RRU经度']<117)&
            (gongcan['RRU纬度']>28)&
            (gongcan['RRU纬度']<34)&
           (gongcan['方位角'].notnull())]

In [ ]:
#只要宏站
gongcan['小区中文名'] = gongcan['小区中文名'].fillna('无')
gongcan = gongcan.loc[~gongcan['小区中文名'].str.contains('W-')]
gongcan = gongcan.loc[~gongcan['小区中文名'].str.contains('R')]
# gongcan = gongcan.loc[~gongcan['小区中文名'].str.contains('M-')]

In [ ]:
gongcan['小区中文名'] = gongcan['小区中文名'].fillna('无')
gongcan.loc[gongcan['小区中文名'].str.contains('M-')].shape

In [ ]:
#打标场景
gongcan = mytools.gisn.points_coverage_merge(gongcan,lon,lat)
print('添加覆盖距离')
gongcan_loc_juli = mytools.gisn.changjing_distance(gongcan,'区域类型',zhishi)

In [ ]:
gongcan_loc_juli.loc[(gongcan_loc_juli['区域类型']=='农村')&(gongcan_loc_juli['工作频段']=='FDD900'),'distance']=1800

In [ ]:
gongcan_loc_juli.head(1)

In [ ]:
gongcan_loc_juli.shape

# 覆盖模型修正

In [ ]:
gongcan_loc_juli['工作频段'] = gongcan_loc_juli['工作频段'].apply(lambda x: 900 if x=='FDD900' else 1800)

gd,band= map(np.log10, (gongcan_loc_juli['天线挂高'],gongcan_loc_juli['工作频段']))

gongcan_loc_juli['覆盖模型距离'] = np.power(10,
                                      (148 - (46.3+33.9*band-13.82*gd-(-3.81))- 3)/(44.9-6.55*gd))*1000

gongcan_loc_juli['30米覆盖模型距离'] = np.power(10,
                                      (148 - (46.3+33.9*band-13.82*np.log10(30)-(-3.81))- 3)/(44.9-6.55*np.log10(30)))*1000

gongcan_loc_juli['使用覆盖半径'] = gongcan_loc_juli['覆盖模型距离'] /gongcan_loc_juli['30米覆盖模型距离']*gongcan_loc_juli['distance']

In [ ]:
def fuck(x):
    if x['区域类型']=='主城区':
        if x['工作频段']==900:
            if x['使用覆盖半径']<200:
                return 200
            elif x['使用覆盖半径']>400:
                return 400
            else:
                return x['distance']
        else:
            if x['使用覆盖半径']<150:
                return 150
            elif x['使用覆盖半径']>350:
                return 350
            else:
                return x['distance']
    elif  x['区域类型']=='县城':
        if x['工作频段']==900:
            if x['使用覆盖半径']<300:
                return 300
            elif x['使用覆盖半径']>430:
                return 430
            else:
                return x['distance']
        else:
            if x['使用覆盖半径']<250:
                return 250
            elif x['使用覆盖半径']>400:
                return 400
            else:
                return x['distance']
    elif  x['区域类型']=='一般城区':
        if x['工作频段']==900:
            if x['使用覆盖半径']<400:
                return 400
            elif x['使用覆盖半径']>1200:
                return 1200
            else:
                return x['distance']
        else:
            if x['使用覆盖半径']<350:
                return 350
            elif x['使用覆盖半径']>800:
                return 800
            else:
                return x['distance']
    elif  x['区域类型']=='乡镇':
        if x['工作频段']==900:
            if x['使用覆盖半径']<430:
                return 430
            elif x['使用覆盖半径']>2200:
                return 2200
            else:
                return x['distance']
        else:
            if x['使用覆盖半径']<400:
                return 400
            elif x['使用覆盖半径']>1000:
                return 1000
            else:
                return x['distance']
    elif  x['区域类型']=='农村':
        if x['工作频段']==900:
            if x['使用覆盖半径']<1200:
                return 1200
            elif x['使用覆盖半径']>2500:
                return 2500
            else:
                return x['distance']
        else:
            if x['使用覆盖半径']<800:
                return 800
            elif x['使用覆盖半径']>1500:
                return 1500
            else:
                return x['distance']
    else:
        return 0

In [ ]:
gongcan_loc_juli['distance_use'] = gongcan_loc_juli.apply(fuck,axis=1)

# 按照天线方向图模型画多边形

In [ ]:
gongcan_loc_juli['所属地区'] = gongcan_loc_juli['所属地区'].fillna('wu')

In [ ]:
gongcan_loc_juli.to_pickle('temp1.data')

In [ ]:
gongcan_loc_juli.shape

In [ ]:
%%writefile temp1.py
from geographiclib.geodesic import Geodesic
from shapely.geometry import Polygon,Point,LineString
import pandas as pd
import geopandas as gpd
import mytools,os ,sys,time,multiprocessing
from multiprocessing import Pool
mx_900 = mytools.othern.pickle_read('G:/mypy/mycloud/jupyter/2_评估/覆盖空洞/20200902评估覆盖空洞-第五版本-覆盖空洞使用天线方向图进行/1-基础数据/44低增益900M0下倾P3_935覆盖模型.data')
mx_f = mytools.othern.pickle_read('G:/mypy/mycloud/jupyter/2_评估/覆盖空洞/20200902评估覆盖空洞-第五版本-覆盖空洞使用天线方向图进行/1-基础数据/F频段模型fad天线.data')
mx_d = mytools.othern.pickle_read('G:/mypy/mycloud/jupyter/2_评估/覆盖空洞/20200902评估覆盖空洞-第五版本-覆盖空洞使用天线方向图进行/1-基础数据/D频段模型fad天线.data')
data_z = pd.read_pickle('temp1.data')
#导入区县图层
def jm_z(x,jd_dp,lon,lat):
    jl = int(x['distance_use'])
    lon = x[lon]
    lat = x[lat]
    fw = x['方位角']
    jd_dp['distance_use'] = jd_dp['电平_正'].apply(lambda x: x/37.052 *jl)
    jd_dp_use = jd_dp.loc[(jd_dp['角度']<=90) | (jd_dp['角度']>=270)]
    jd_dp_use['角度'] = pd.to_numeric(jd_dp_use['角度'] )
    jd_dp_use['角度2'] = jd_dp_use['角度'].apply(lambda x: x-0+fw if x-0+fw<360 else x-0+fw-360 )
    points=[]
    dict_use = dict()
    for jiao , jl in zip(jd_dp_use['角度2'],jd_dp_use['distance_use']):
        res = Geodesic.WGS84.Direct(lat, lon, jiao, jl)
        if int(jiao)%10==0:
            dict_use[jiao]=res['lon2'], res['lat2']
        points.append((res['lon2'], res['lat2']))
    points.append(points[0])
    pol = Polygon(points)
    return pol
def fcgo(data_t):
    data_t['geometry'] = data_t.apply(lambda x:jm_z(x,mx_900,'RRU经度','RRU纬度'),axis=1)

    return data_t
if __name__ == '__main__':
    threads = []
    with Pool(processes=6) as pool:         # start 4 worker processes
        aa = pd.DataFrame()
        for name , data_t in data_z.groupby('所属地区'):
            print(name,'开始')
            result = pool.apply_async(fcgo, (data_t,)) # evaluate "f(10)" asynchronously in a single process
            threads.append(result)
        pool.close()#防止将更多任务提交到池中。完成所有任务后，工作进程将退出。
        # 调用join之前，先调用close函数，否则会出错。执行完close后不会有新的线程加入到pool,
        # join函数等待所有子线程结束
        pool.join()
        for result_t in threads:
            data_tp = result_t.get()
            aa = aa.append(data_tp)
            print(data_tp.shape,aa.shape)
        print(aa.shape)
        aa.to_pickle('res.pkl')

In [ ]:
mytools.dog()

In [ ]:
# %run temp.py
# if os.path.exists('temp.py'):  # 如果文件存在
#     os.remove('temp.py') # 删除文件

In [ ]:
grid_changjing = pd.read_pickle('res.pkl')
# if os.path.exists('res.pkl'):  # 如果文件存在
#     os.remove('res.pkl') # 删除文件
# if os.path.exists('temp1.data'):  # 如果文件存在
#     os.remove('temp1.data') # 删除文件

In [ ]:
gongcan_loc_juli = grid_changjing.copy()

In [ ]:
gdfdata = gpd.GeoDataFrame(gongcan_loc_juli, crs={'init': 'epsg:4326'},geometry='geometry')
gdfdata['type'] = gdfdata.type
gdfdata.groupby('type').count()

In [ ]:
# gdfdata.to_csv('G:/mypy/mycloud/jupyter/2_评估/覆盖空洞/20200902评估覆盖空洞-第四版本-覆盖空洞使用天线方向图进行/2-输出数据/天线方向图模型hata修正_东新.csv',encoding='gbk')

# gdfdata.to_pickle('G:/mypy/mycloud/jupyter/2_评估/覆盖空洞/20200902评估覆盖空洞-第四版本-覆盖空洞使用天线方向图进行/2-输出数据/天线方向图模型hata修正_东新.data')

In [ ]:
use_data = gdfdata.loc[gdfdata['归属区县']=='洪山区']

In [ ]:
use_data.head(12).plot(figsize=[12,12])

In [ ]:
use_data.head(1)

In [ ]:
lon = 114.33871
lat = 30.50173
ji = 1000

In [ ]:
x = 270

In [ ]:
def fw_zl(fw=360):
    if x < 90 :
        jg = list(range(x%10,x+100,10))
        jg2 = list(range(360-(90-x),360,10))
        jg = jg2 + jg
    elif x > 270:
        jg = list(range(x-90,x+(360-x),10))
        jg2= list(range((90-(360-x))%10,90-(360-x)+10,10))
        jg = jg + jg2
    else:
        jg = list(range(x-90,x+100,10))
    return jg

In [ ]:
points=[]
dict_use = dict()
for jiao in [x for x in jg]:
    res = Geodesic.WGS84.Direct(lat, lon, jiao, jl)
    if jiao%10==0:
        dict_use[jiao]=res['lon2'], res['lat2']
    points.append((res['lon2'], res['lat2']))
points.append(points[0])

In [ ]:
Polygon(points)

In [ ]:
dict_use

In [ ]:
pol = Polygon(points)

In [ ]:
lins=[]
for key,data_t in dict_use.items():
    lon2,lat2=data_t
    lins.append((lon,lat))
    lins.append((lon2,lat2))

In [ ]:
lins = [LineString([(lon, lat), (data_t[0], data_t[1])]) for key,data_t in dict_use.items()]

In [ ]:
data_z = pd.read_pickle('temp1.data')

In [ ]:
mx_900 = mytools.othern.pickle_read('G:/mypy/mycloud/jupyter/2_评估/覆盖空洞/20200902评估覆盖空洞-第五版本-覆盖空洞使用天线方向图进行/1-基础数据/44低增益900M0下倾P3_935覆盖模型.data')


In [ ]:
mx_900

In [ ]:
r

In [ ]:
use_data = use_data.head(12)

In [ ]:
mx_900.shape

In [ ]:
use_data['geometry'] = use_data.apply(lambda x:jm_z(x,mx_900,'RRU经度','RRU纬度','方位角'),axis=1)

In [ ]:
use_data.head(12).plot(figsize=[12,12])

In [ ]:
use_data2 = use_data.explode()

In [ ]:
#导入建筑物
wuhan_jz = gpd.read_file('g:/1-规划/图层/湖北建筑/湖北建筑/湖北建筑/武汉.tab',encoding='gbk')

In [ ]:
wuhan_jz.crs

In [ ]:
wuhan_jz = wuhan_jz.to_crs({'init': 'epsg:4326'})

In [ ]:
wuhan_jz = wuhan_jz[['OBJECTID','建筑物高度','geometry']]

In [ ]:
use_data2.columns = ['cgi', 'cellname', 'city', 'lon', 'lat', 'gaodu', 'fw', 'band',
       'qux', 'vender', '区域类型', 'distance', '覆盖模型距离', '30米覆盖模型距离', '使用覆盖半径',
       'distance_use', 'type_2', 'geometry']

In [ ]:
use_data3 = use_data2.reset_index()

In [ ]:
res_sjoin = gpd.sjoin(use_data3,wuhan_jz,how='inner')

In [ ]:
jianzhu = wuhan_jz.loc[wuhan_jz['OBJECTID']==398879]

In [ ]:
fugai =use_data3.head(1)

In [ ]:
fugai.plot(figsize=[12,12])

In [ ]:
jianzhu.plot(figsize=[12,12])

In [ ]:
jiaoji = gpd.overlay(fugai,jianzhu)

In [ ]:
import numpy as np

In [ ]:
zhuan = (6371000*np.pi/180)

In [ ]:
Point(114.33871,30.50173).distance(jiaoji['geometry'][0])*zhuan

In [ ]:
jianzhu

In [ ]:
import simplekml

In [ ]:
kml = simplekml.Kml()

In [ ]:
grid_red = kml.newpoint(name='a',coords=[(114.33871,30.50173)])

In [ ]:
kml.save('d:/a.kml') 

In [ ]:
kml = simplekml.Kml()
kml.newpolygon(name='grid',outerboundaryis=list(jiaoji['geometry'][0].exterior.coords))
kml.save('d:/ppp.kml') 

In [ ]:
lie = LineString(((114.33871,30.50173),(114.335979, 30.50040),(114.335070, 30.50240)))

In [ ]:
lie

In [ ]:
lie.union(jiaji_use)

In [ ]:
res = lie.difference(jiaji_use)

In [ ]:
for aa in res:
    print(type(aa))

In [ ]:
def jm_zl(x,jd_dp,lon,lat,fw):
    jl = int(x['distance_use'])
    lon = x[lon]
    lat = x[lat]
    fw = x[fw]
    jd_dp['distance_use'] = jd_dp['电平_正'].apply(lambda x: x/37.052 *jl)
    jd_dp_use = jd_dp.loc[(jd_dp['角度']<=90) | (jd_dp['角度']>=270)]
    jd_dp_use['角度'] = pd.to_numeric(jd_dp_use['角度'] )
    jd_dp_use['角度2'] = jd_dp_use['角度'].apply(lambda x: x-0+fw if x-0+fw<360 else x-0+fw-360 )
    poy=[]
    dict_use = dict()
    a=0
    for jiao1,jiao2 , jl in zip(jd_dp_use['角度'],jd_dp_use['角度2'],jd_dp_use['distance_use']):
        res = Geodesic.WGS84.Direct(lat, lon, jiao2, jl)
        points=[]
        if int(jiao1)%10==0:
            print('可以',jiao1,jiao2)
            points.append((lon, lat))
            points.append((res['lon2'], res['lat2']))
            poy.append(LineString(points))
    res_m = MultiLineString(poy)
    return res_m

In [ ]:
0%10

In [ ]:
use_data['geometry'] = use_data.apply(lambda x:jm_zl(x,mx_900,'RRU经度','RRU纬度','方位角'),axis=1)

In [ ]:
use_data.head(1)

In [ ]:
use_data.head(1).plot(figsize=[12,12])

In [ ]:
sxingx = use_data['geometry'][55]

In [ ]:
res = sxingx.difference(jiaji_use)

In [ ]:
res.

In [ ]:
res.boundary

In [ ]:
linshi_2 = []
for ll in res :
    print(ll.coords)
    if (114.33871, 30.50173) in [x for x in ll.coords[:]]:
        linshi_2.append(ll.coords[-1])


In [ ]:
Polygon(linshi_2)

In [ ]:
from shapely.ops import unary_union

buffer_size = 0.00000000001
lines_buffered = [line.buffer(buffer_size) for line in lins]
quadrats = unary_union(lines_buffered)
multipoly = pol.difference(quadrats)
multipoly

In [ ]:
gdfdata.to_pickle('G:/1-规划/评估/覆盖空洞/20200907第五版/2-输出内容/xiaoqufugaimoxing.data')

# 覆盖空洞计算

In [ ]:
gongcan_s = gdfdata.copy()

In [ ]:
# gongcan_s['归属区县'] = gongcan_s['归属区县'].fillna('wu')

去掉农村覆盖到场景边界内的区域

In [ ]:
#---------------------------规划图层缩小50%的距离---------------------------

guihua = gpd.read_file('G:/1-规划/图层/全省区域规划图层20200110/全省区域规划图层20200110.TAB',encoding='gbk')
guihua['区域类型2'] = guihua['区域类型'].replace({'3A景区':'农村','4A景区':'农村','5A景区':'农村','景区':'农村','热点':'农村'})
guihua_use = guihua.loc[guihua['区域类型2']!='农村']
guihua_yiban = mytools.gisn.changjing_distance(guihua_use,'区域类型2','区域类型')
guihua_yiban['distance2'] = 0-guihua_yiban['distance']/2
guihua_res = mytools.gisn.buffer_df(guihua_yiban,'distance2')
guihua_res.columns =['city', 'region', 'name', 'type', 'area', 'geometry', 'type2', 'distance','distance2']

# guihua_res.to_file('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/2-规划图层缩小场景1800覆盖半径的一半距离/规划图层剔除农村1800场景一半.shp',encoding='utf-8')
# guihua_res= gpd.read_file('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/2-规划图层缩小场景1800覆盖半径的一半距离/规划图层剔除农村1800场景一半.shp',encoding='utf-8')

In [ ]:
guihua_res = guihua_res.loc[~guihua_res.is_empty]

In [ ]:
guihua_res.plot(figsize=[11,11])

In [ ]:
#---------------------------农村覆盖区域---------------------------
gongcan_s_nc = gongcan_s.loc[gongcan_s['区域类型']=='农村']
gongcan_s_nc['所属地区'] = gongcan_s_nc['所属地区'].fillna('wu')

#全网农村小区覆盖按照地市合并
gongcan_s_nc_city = gongcan_s_nc.dissolve(by = '所属地区')

In [ ]:
#只是留下geo去除索引，索引是地市
gongcan_s_nc_city_use = gongcan_s_nc_city[['geometry']].reset_index()

gongcan_s_nc_city_use.columns=['city','geometry']

In [ ]:
guihua_res = guihua_res.dropna(subset=['geometry'])

In [ ]:
#在全网农村覆盖区域中去掉农村覆盖到其他区域的
nongcun = gpd.overlay(gongcan_s_nc_city_use,guihua_res,how='difference')

In [ ]:
#农村区域按照地市合并
nongcun_city = nongcun.dissolve(by = 'city')

nongcun_city =nongcun_city.reset_index()
# nongcun_city.to_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/3-小区覆盖区域/农村场景小区覆盖区域.data')

# nongcun_city = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/3-小区覆盖区域/农村场景小区覆盖区域.data')

nongcun_city.plot(figsize=[11,11])

In [ ]:
# 1-按照区县合并覆盖区域
gongcan_s_feinc = gongcan_s.loc[gongcan_s['区域类型']!='农村']

#小区覆盖区域图层只保留两列
gongcan_s_feinc = gongcan_s_feinc[['所属地区','geometry']]

gongcan_s_feinc.columns = ['city','geometry']

gongcan_s_z = gongcan_s_feinc.append(nongcun_city)

#小区覆盖区域按照地市合并
gongcan_s_z = gongcan_s_z.dissolve(by = 'city')

# gongcan_s_z.to_pickle('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/3-小区覆盖区域/农村和非农村合并全网覆盖图层按照地市合并.data')

# gongcan_s_z.to_file('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/3-小区覆盖区域/农村和非农村合并全网覆盖图层按照地市合并.shp',encoding='utf-8')

mytools.dog()

2-求出覆盖空洞原始

In [ ]:
tu = mytools.gisn.maps()

In [ ]:
city_tu_diff = gpd.overlay(tu,gongcan_s_z,how='difference')

city_tu_diff.to_pickle('G:/1-规划/评估/覆盖空洞/20200907第五版/2-输出内容/原始空洞.data')

city_tu_diff.to_file('G:/1-规划/评估/覆盖空洞/20200907第五版/2-输出内容/原始空洞.shp',encoding='utf-8')

In [ ]:
city_tu_diff.plot(figsize=(11,11),color='gray')

In [ ]:
mytools.dog()

# -------------剔除水系-------------

In [ ]:
city_tu_diff.head(1)

In [ ]:
# city_tu_diff = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200907第五版/2-输出内容/原始空洞.data')

In [ ]:
# hb_sx_city=pd.read_pickle('G:/1-规划/图层/水域/水域按照地市划分.data')

In [ ]:
hb_sx_city1 = pd.read_pickle('G:/1-规划/图层/水域/水域按照地市合并.data')

In [ ]:
kongdong_t = gpd.overlay(city_tu_diff,hb_sx_city1,how='difference')

In [ ]:
%%writefile temp2.py
from geographiclib.geodesic import Geodesic
from shapely.geometry import Polygon,Point,LineString
import pandas as pd
import geopandas as gpd
import mytools,os ,sys,time,multiprocessing
from multiprocessing import Pool
hb_sx_city1 = pd.read_pickle('G:/1-规划/图层/水域/水域按照地市合并.data')
city_tu_diff = pd.read_pickle('G:/1-规划/评估/覆盖空洞/20200907第五版/2-输出内容/原始空洞.data')
citys = ['十堰','咸宁','天门','孝感', '宜昌','恩施','武汉', '江汉', '潜江', '荆州', '荆门', '襄阳', '鄂州', '随州', '黄冈', '黄石']

def fcgo(city_tu_diff_t,hb_sx_city1_t):
    kongdong_t = gpd.overlay(city_tu_diff_t,hb_sx_city1_t,how='difference')
    return kongdong_t

    return data_t
if __name__ == '__main__':
    threads = []
    with Pool(processes=6) as pool:         # start 4 worker processes
        aa = pd.DataFrame()
        for name in citys:
            city_tu_diff_t = city_tu_diff.loc[city_tu_diff['city']==name]
            hb_sx_city1_t = hb_sx_city1.loc[hb_sx_city1['city']==name]
            print(name,'开始')
            result = pool.apply_async(fcgo, (city_tu_diff_t,hb_sx_city1_t,)) # evaluate "f(10)" asynchronously in a single process
            threads.append(result)
        pool.close()#防止将更多任务提交到池中。完成所有任务后，工作进程将退出。
        # 调用join之前，先调用close函数，否则会出错。执行完close后不会有新的线程加入到pool,
        # join函数等待所有子线程结束
        pool.join()
        for result_t in threads:
            data_tp = result_t.get()
            aa = aa.append(data_tp)
            print(data_tp.shape,aa.shape)
        print(aa.shape)
        aa.to_pickle('空洞.pkl')

In [ ]:
%run temp.py
if os.path.exists('temp2.py'):  # 如果文件存在
    os.remove('temp2.py') # 删除文件

In [ ]:
grid_changjing = pd.read_pickle('空洞.pkl')
if os.path.exists('res.pkl'):  # 如果文件存在
    os.remove('res.pkl') # 删除文件
if os.path.exists('temp1.data'):  # 如果文件存在
    os.remove('temp1.data') # 删除文件

In [ ]:
kongdong_t.plot(figsize=(11,11),color='gray')

In [ ]:
kongdong_t.columns

In [ ]:
grid_changjing.to_pickle('G:/1-规划/评估/覆盖空洞/20200907第五版/2-输出内容/剔除水系后的空洞.data')

In [ ]:
grid_changjing.to_file('G:/1-规划/评估/覆盖空洞/20200907第五版/2-输出内容/剔除水系后的空洞.shp',encoding='utf-8')

In [ ]:
hb_sx_city1 = pd.read_pickle('G:/1-规划/图层/水域/水域按照地市合并.data')
kongdong = gpd.overlay(city_tu_diff,hb_sx_city1,how='difference')
mytools.sound()

kongdong.to_pickle('G:/1-规划/评估/覆盖空洞/20200907第五版/2-输出内容/剔除水系后的空洞.data')
# kongdong.to_file('G:/1-规划/评估/覆盖空洞/20200712第四版本/2-程序数据出-过程/6-剔除水系后空洞/剔除水系后的空洞.shp',encoding='utf-8')
kongdong.plot(figsize=(11,11),color='coral')
mytools.sound()

In [ ]:
kongdong.to_pickle('G:/1-规划/评估/覆盖空洞/20200907第五版/2-输出内容/剔除水系后的空洞.data')

# -------------空洞有效数据整理-------------

# 1-拆分成多个空洞-打标场景 划分大中小微

In [ ]:
# 1-拆分成多个空洞-打标场景 划分大中小微

In [ ]:
kongdong.head(1)

kongdong['type'] =  kongdong.type

kongdong.groupby('type').count()

kongd_use = kongdong.loc[kongdong['type']=='Polygon'].append(kongdong.loc[kongdong['type']=='MultiPolygon'].explode())

kongd_use.shape

kongd_mianji = mytools.gisn.add_area(kongd_use)

kongd_mianji['type'] =  kongd_mianji.type

kongd_mianji =kongd_mianji.reset_index(drop=True)

# 打标场景

kongd_mianji[['lon','lat']] = kongd_mianji.apply(lambda x:pd.Series(x['geometry'].centroid.coords[:][0]),axis=1)#

result = mytools.gisn.points_coverage_merge(kongd_mianji,'lon','lat')

result.shape

#添加id编号
result_id = result.reset_index(drop=True)
result['id'] = [ 'kd_{}'.format(str(name)) for name in result_id.index+1]

result['制式'] = 'FDD1800'
result.loc[result['区域类型']=='农村','制式']='FDD900'

result = mytools.gisn.changjing_distance(result,'区域类型','制式')

result['标准面积'] = np.pi* result['distance']* result['distance']
result['空洞面积比例'] = result['面积']/result['标准面积']

def fuck_data(x):
    if x['空洞面积比例']>=1:
        return '大'
    elif x['空洞面积比例']>=1/3:
        return '中'
    elif x['空洞面积比例']>=1/6:
        return '小'
    else:
        return '微'

result['空洞级别'] = result.apply(fuck_data,axis=1)

result.head(1)

result.to_pickle('G:/1-规划/评估/覆盖空洞/20200907第五版/2-输出内容/覆盖空添加场景区分大中小微.data')

# result.reindex(columns=['id','city','面积','lon','lat', '区域类型','制式','distance','标准面积','空洞面积比例','空洞级别']).to_excel(
#     'G:/1-规划/评估/覆盖空洞/20200712第四版本/3-拆分后的空洞/覆盖空19481个添加场景区分大中小微.xlsx',index=False)

# result.columns = ['city', 'type', 'geometry', 'area', 'lon', 'lat', 'regiontype', 'id', 'frequency ',
#        'distance', 'scene_area', 'kd_area_specific', 'kd_class']

In [ ]:
result.columns = ['city', 'qx', 'geometry', 'type', 'area', 'lon', 'lat', 'qylx', 'id', 'zhishi',
       'distance', 'bzarea', 'kongd_area_zhanbi', 'jibie']

In [ ]:
result.to_file('G:/1-规划/评估/覆盖空洞/20200907第五版/2-输出内容/覆盖空添加场景区分大中小微.shp',encoding='utf-8')

In [ ]:
result.head(1)

In [ ]:
result.groupby('jibie').count()

In [ ]:
data_fg = gdfdata.explode().reset_index()

In [ ]:
f = mytools.othern.file_name_paths('G:/1-规划/图层/湖北建筑/湖北建筑/湖北建筑/',find='荆州.tab')

In [ ]:
print(data_fg['geo'][0])

In [ ]:
a = Point(1, 1).buffer(1.5)
b = Point(2, 1).buffer(1.5)
c = Point(4, 2)

In [ ]:
a.distance(c)

In [ ]:
Point(1, 3).distance(b)

In [ ]:
b = Polygon([(1, 0), (3, 0), (3, 3),(1, 0)])

In [ ]:
c = Point(0,0)

In [ ]:
b.distance(c)

In [ ]:
data = pd.read_clipboard()

In [ ]:
data['lon'] = np.abs(113.887821 - data['lon'])
data['lat'] = np.abs(34.020434 - data['lat'])
data['sum'] = data['lon']+data['lat']

In [ ]:
data.sort_values('sum')

In [ ]:
a.unary_union(b)

In [ ]:
b

In [ ]:
b = Polygon([(112.1473500278309,30.12404993354895), (112.1474500409121,30.12404993837585), (112.153296449951,30.11625041859126), (112.1523758624733,30.12916740380928), (112.1473500278309,30.12404993354895)])

In [ ]:
a = data_fg['geo'][0]

In [ ]:
a.difference(b)

In [ ]:
sjoin_data = gpd.GeoDataFrame()
for name in f:
    tuceng_t = gpd.read_file(name,encoding = 'gbk')
    tuceng_t = tuceng_t.to_crs({'init': 'epsg:4326'})
    linshi_t = gpd.sjoin(data_fg,tuceng_t)
    linshi_t = linshi_t[[ '小区CGI(*)','建筑物高度','SHAPE_Area']]
    sjoin_data = sjoin_data.append(linshi_t)
    print(name,'ok')

In [ ]:
sjoin_data['volume'] = sjoin_data['建筑物高度'] * sjoin_data['SHAPE_Area']

sjoin_data_volume_h = sjoin_data.groupby('小区CGI(*)').agg({'volume':'sum','SHAPE_Area':'sum'})

sjoin_data_volume_h['h_use'] = sjoin_data_volume_h['volume'] / sjoin_data_volume_h['SHAPE_Area']

sjoin_data_linshi = sjoin_data_volume_h.reset_index()

gongcan_zhangao = gongcan_loc_juli.merge(sjoin_data_linshi[['小区CGI(*)','h_use']],how='left',on='小区CGI(*)')

gongcan_zhangao['高度建筑比'] = (gongcan_zhangao['天线挂高']-gongcan_zhangao['h_use'])/gongcan_zhangao['天线挂高']

In [ ]:
jd_dp.to_clipboard()

In [ ]:
def jm(x):
    jl = int(x['距离'])
    lon = x['lon']
    lat = x['lat']
    fw = x['方位角']
    jd_dp['距离'] = jd_dp['电平_正'].apply(lambda x: x/37.052 *jl)
    jd_dp_use = jd_dp.loc[(jd_dp['角度']<=90) | (jd_dp['角度']>=270)]
    jd_dp_use['角度'] = pd.to_numeric(jd_dp_use['角度'] )
    jd_dp_use['角度2'] = jd_dp_use['角度'].apply(lambda x: x-0+fw if x-0+fw<360 else x-0+fw-360 )
    points=[]
    dict_use = dict()
    for jiao , jl in zip(jd_dp_use['角度2'],jd_dp_use['距离']):
        res = Geodesic.WGS84.Direct(lat, lon, jiao, jl)
        if int(jiao)%10==0:
            dict_use[jiao]=res['lon2'], res['lat2']
        points.append((res['lon2'], res['lat2']))
    points.append(points[0])
    pol = Polygon(points)
    lins=[]
    for key,data_t in dict_use.items():
        lon2,lat2=data_t
        lins.append((lon,lat))
        lins.append((lon2,lat2))
    lins = [LineString([(lon, lat), (data_t[0], data_t[1])]) for key,data_t in dict_use.items()]
    buffer_size = 0.00000000001
    lines_buffered = [line.buffer(buffer_size) for line in lins]
    quadrats = unary_union(lines_buffered)
    multipoly = pol.difference(quadrats)
    return multipoly

In [ ]:
def jm(x,jd_dp,lon,lat):
    jl = int(x['distance_use'])
    lon = x[lon]
    lat = x[lat]
    fw = x['方位角']
    jd_dp['distance_use'] = jd_dp['电平_正'].apply(lambda x: x/37.052 *jl)
    jd_dp_use = jd_dp.loc[(jd_dp['角度']<=90) | (jd_dp['角度']>=270)]
    jd_dp_use['角度'] = pd.to_numeric(jd_dp_use['角度'] )
    jd_dp_use['角度2'] = jd_dp_use['角度'].apply(lambda x: x-0+fw if x-0+fw<360 else x-0+fw-360 )
    points=[]
    dict_use = dict()
    for jiao , jl in zip(jd_dp_use['角度2'],jd_dp_use['distance_use']):
        res = Geodesic.WGS84.Direct(lat, lon, jiao, jl)
        if int(jiao)%10==0:
            dict_use[jiao]=res['lon2'], res['lat2']
        points.append((res['lon2'], res['lat2']))
    points.append(points[0])
    pol = Polygon(points)
    lins=[]
    for key,data_t in dict_use.items():
        lon2,lat2=data_t
        lins.append((lon,lat))
        lins.append((lon2,lat2))
    lins = [LineString([(lon, lat), (data_t[0], data_t[1])]) for key,data_t in dict_use.items()]
    buffer_size = 0.00000000001
    lines_buffered = [line.buffer(buffer_size) for line in lins]
    quadrats = unary_union(lines_buffered)
    multipoly = pol.difference(quadrats)
    return multipoly

In [ ]:
# data = data.set_index('cgi')

In [ ]:
data['geolin'] = data.apply(jm,axis=1)

In [ ]:
gdfdata = gpd.GeoDataFrame(data, crs={'init': 'epsg:4326'},geometry='geolin')

gdfdata['type'] = gdfdata.type

gdfdata.groupby('type').count()

In [ ]:
data_fg = gdfdata.explode().reset_index(drop=1)

In [ ]:
gdfdata.loc[1:1,]

In [ ]:

gdfdata.loc[1:1,].plot(figsize =[12,12])

In [ ]:
data_fg.head(8).plot(figsize =[12,12])

# 进行覆盖模型修正-按照hata覆盖模型和站高进行修正

In [ ]:
jd_dp['电平']

In [ ]:
jd_dp['距离_模型'] = np.power(10,(148-jd_dp['电平'] - (46.3+33.9*np.log10(1800)-13.82*np.log10(30)-(-3.81))- 3)/(44.9-6.55*30))*1000

In [ ]:
jd_dp.to_clipboard()

In [ ]:
jd_dp['距离'] = jd_dp['电平_正'].apply(lambda x: x/37.052 *927)